In [ ]:
############################################################
# Function to count the number of images per car make
############################################################

from collections import Counter
def count_labels(dataset, make = True):
	# Extracts the labels from the dataset
	if make:
		list = [make for _, make, _ in dataset]
	else:
		list = [model for _, _, model in dataset]

	# Counts the occurrences of each label
	dataset_count = Counter(list)

	# Transform the labels into the string using get_names function
	if make:
		dataset_count_strings = {tuple(get_names(mat_file, [label, 0]))[0]: count for label, count in dataset_count.items()}
	else:
		dataset_count_strings = {tuple(get_names(mat_file, [0, label]))[1]: count for label, count in dataset_count.items()}

	# Print the labels and their counts
	print("Dataset labels:", dataset_count_strings)

	# Sort the labels by alphabetical order
	sorted_labels = sorted(dataset_count_strings.items(), key=lambda x: x[0])
	labels, counts = zip(*sorted_labels)

	# Create the plot
	plt.figure(figsize=(15, 5))
	plt.bar(labels, counts)
	plt.xticks(rotation=90)
	plt.ylabel("Number of images")
	plt.title("Number of images per car make in the training dataset")
	plt.show()

	return dataset_count_strings



train_counts = count_labels(train_dataset)
val_counts = count_labels(val_dataset)
test_counts = count_labels(test_dataset)

In [ ]:
#HERE I TRY USING THE GITHUB LINK TECHNIQUE
from torchvision.models import Inception_V3_Weights
# Load the model
net = torchvision.models.inception_v3(weights=Inception_V3_Weights.IMAGENET1K_V1)

# Change the number of output features
MODELS_NUM = 431
MAKE_NUM = 163

num_features = net.fc.in_features
net.fc = nnpo.Linear(num_features, MAKE_NUM)

if torch.cuda.device_count() > 1:
    net = torch.nn.DataParallel(net)

net.cuda()

criterion = torch.nn.CrossEntropyLoss()
criterion.cuda()

optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
my_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
dloaders = {'train': train_loader, 'valid': val_loader}



In [ ]:
from torch.utils.tensorboard import SummaryWriter


def train_model(dataloders, model, criterion, optimizer, scheduler, num_epochs=10000):
    logger = SummaryWriter()
    since = time.time()
    best_acc = 0.0
    dataset_sizes = {'train': len(dataloders['train'].dataset),
                     'valid': len(dataloders['valid'].dataset)}
    print("train and valid sizes %d %d" % (len(dataloders['train'].dataset), len(dataloders['valid'].dataset)))

    for epoch in range(num_epochs):
        epoc_time = time.time()
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        for phase in ['train', 'valid']:
            if phase == 'train':
                scheduler.step()
                model.train()
            else:
                model.eval()

            running_loss = 0.0

            if phase == 'train':
                train_running_corrects = 0
            else:
                valid_running_corrects = 0

            # Use tqdm to create a progress bar
            with tqdm(total=len(dataloders[phase]), desc=f'{phase} Epoch {epoch + 1}/{num_epochs}', unit='batch') as pbar:
                for inputs, label_make, label_model in dataloders[phase]:
                    t1 = time.time()
                    labels = label_make
                    inputs, labels = inputs.cuda(), labels.cuda()

                    optimizer.zero_grad()

                    t2 = time.time()
                    outputs = model(inputs)
                    t3 = time.time()

                    if phase == 'train':
                        loss1 = criterion(outputs[0], labels)
                        loss2 = criterion(outputs[1], labels)
                        loss = loss1 + loss2  # Future work, change to e.g. loss1 + 0.4 x loss2
                        preds = torch.max(outputs[0], 1)[1]
                        train_running_corrects += torch.sum(preds == labels.data)
                    else:
                        loss = criterion(outputs, labels)
                        _, preds = torch.max(outputs.data, 1)
                        valid_running_corrects += torch.sum(preds == labels.data)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                    t4 = time.time()

                    running_loss += loss.item()
                    pbar.update(1)  # Update the progress bar

            if phase == 'train':
                train_epoch_loss = running_loss / dataset_sizes[phase]
                train_epoch_acc = train_running_corrects.double() / dataset_sizes[phase] * 100
            else:
                valid_epoch_loss = running_loss / dataset_sizes[phase]
                valid_epoch_acc = valid_running_corrects.double() / dataset_sizes[phase] * 100

            if phase == 'valid' and valid_epoch_acc > best_acc:
                best_acc = valid_epoch_acc
                best_model_wts = model.state_dict()
                torch.save({
                    'epoch': epoch,
                    'epochTrainLoss': train_epoch_loss,
                    'epochValidLoss': valid_epoch_loss,
                    'epochTrainAcc': train_epoch_acc,
                    'epochValidAcc': valid_epoch_acc,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                }, os.path.join(DATA_PATH, "Inception_classification.pth"))

        epocTotalTime = time.time() - epoc_time
        epocLoadDataTime = t2 - t1
        epocForwardTime = t3 - t2
        epocLossBackwardTime = t4 - t3
        print('Epoch [{}/{}] train loss: {:.4f} acc: {:.4f}% '
              'valid loss: {:.4f} acc: {:.4f}% Time: {:.0f}s train corr: {:d}  valid corr: {:d}  '.format(
                epoch, num_epochs - 1,
                train_epoch_loss, train_epoch_acc,
                valid_epoch_loss, valid_epoch_acc,
                (time.time() - epoc_time),
                train_running_corrects,
                valid_running_corrects
              ))
        logger.add_scalar('Train/epocLoss', train_epoch_loss, epoch)
        logger.add_scalar('Train/accuracy', train_epoch_acc, epoch)
        logger.add_scalar('Valid/epocLoss', valid_epoch_loss, epoch)
        logger.add_scalar('Valid/accuracy', valid_epoch_acc, epoch)
        logger.add_scalar('Train/epocTotalTime', epocTotalTime, epoch)
        logger.add_scalar('Train/epocLoadDataTime', epocLoadDataTime, epoch)
        logger.add_scalar('Train/epocForwardTime', epocForwardTime, epoch)
        logger.add_scalar('Train/epocLossBackward', epocLossBackwardTime, epoch)

    print('Best val Acc: {:4f}'.format(best_acc))
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    return model

In [ ]:
num_epoches = 10000
start_time = time.time()

model = train_model(dloaders, net, criterion, optimizer, my_scheduler, num_epochs=num_epoches)
print('Training time: {:10f} minutes'.format((time.time()-start_time)/60))

In [ ]:
# toy exemple to understand the cross entropy loss

import torch
import torch.nn as nn

# Supponiamo di avere un batch di dimensione 3 e 5 classi
logits = torch.tensor([[0.0, 2.0, 0.0, 4.0, 1.0],
                       [1.0, 0.0, 0.0, 1.0, 4.0],
                       [4.0, 1.0, 2.0, 0.0, 1.0]], requires_grad=True)

# Etichette di classe vere per ciascun esempio nel batch
labels = torch.tensor([3, 4, 0])

print(logits[0], logits[1], logits[2])  # Stampa i logit per ciascun batch

# Definisci la funzione di perdita
criterion = nn.CrossEntropyLoss()

# Calcola la perdita
loss = criterion(logits, labels)

print(loss)

### PRETRAINED NET

In [ ]:
MODELS_NUM = 431
MAKE_NUM = 163
car_make_bool = True   ## TODO Remove this line
checkpoint_name = "checkpoint.pth.tar"
checkpoint_path = os.path.join(DATA_PATH, checkpoint_name)

# Function to save the checkpoint
def save_checkpoint(state, filename):
    torch.save(state, filename)

# Function to load the checkpoint
def load_checkpoint(model, optimizer, scheduler, filename):
    if os.path.isfile(filename):
        print(f"Loading checkpoint '{filename}'")
        checkpoint = torch.load(filename)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        epoch = checkpoint['epoch']
        best_acc = checkpoint['best_acc']
        print(f"Loaded checkpoint '{filename}' (epoch {epoch})")
        return model, optimizer, scheduler, epoch, best_acc
    else:
        print(f"No checkpoint found at '{filename}'")
        return model, optimizer, scheduler, 0, 0.0


# Function to calculate the accuracy of the model
def accuracy(outputs, labels):
	_, preds = torch.max(outputs, dim=1)   # Get the prediction label for the batch
	return torch.tensor(torch.sum(preds == labels).item() / len(preds))  #sees how many predictions are correct in the batch

####TODO Check if the function is correct
# Function to predict the class of an image
def predict_image(image_path, model, transform):
	image = Image.open(image_path).convert("RGB")
	image = transform(image).unsqueeze(0).to(device)
	outputs = model(image)
	_, preds = torch.max(outputs, 1)
	return preds.item()



In [ ]:

# Function to train the model
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=25, start_epoch=0, best_acc=0.0):
    best_model_wts = None

    for epoch in range(start_epoch, start_epoch + num_epochs):
        print('Epoch {}/{}'.format(epoch, start_epoch + num_epochs - 1))
        print('-' * 10)
        # if epoch > 20:
        #     # Unfreeze the layers after 20 epochs
        #     for param in model.parameters():
        #         param.requires_grad = True

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
                dataloader = train_loader
            else:
                model.eval()   # Set model to evaluation mode
                dataloader = val_loader

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data
            iterator = tqdm(dataloader, desc=phase)
            for batch_x, batch_car_make, batch_car_model in iterator:
                batch_x = batch_x.to(device)
                if car_make_bool:
                    labels = batch_car_make.to(device)
                else:
                    labels = batch_car_model.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward pass
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(batch_x)   # batch size x num_classes
                    loss = criterion(outputs, labels)

                    # Backward pass + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * batch_x.size(0)  #type float like loss.detach().cpu().numpy()
                running_corrects += accuracy(outputs, labels)

            if phase == 'train':
                train_loss = running_loss / len(train_dataset)
                train_acc = running_corrects.double() / len(train_dataset)
            else:
                val_loss = running_loss / len(val_dataset)
                val_acc = running_corrects.double() / len(val_dataset)

                # Update the learning rate
                scheduler.step()

                # Save the best model
                if val_acc > best_acc:
                    best_acc = val_acc
                    best_model_wts = model.state_dict()
                    # Save the model only if it is the best model
                    save_checkpoint({
                        'epoch': epoch,
                        'model_state_dict': model.state_dict(),
                        'best_model_wts': best_model_wts,
                        'optimizer_state_dict': optimizer.state_dict(),
                        'scheduler_state_dict': scheduler.state_dict(),
                        'best_acc': best_acc,
                        'train_loss': train_loss,
                        'val_loss': val_loss,
                        'train_acc': train_acc,
                        'val_acc': val_acc,
                    }, checkpoint_path)
                    print("Model saved")

            print(f'{phase} Loss: {train_loss if phase == "train" else val_loss:.4f} Acc: {train_acc if phase == "train" else val_acc:.4f}')

        print()

    print('Best val Acc: {:4f}'.format(best_acc))

    # Load the best model weights
    model, optimizer, scheduler, start_epoch, best_acc = load_checkpoint(model, optimizer, scheduler, checkpoint_path)

    return model

# Function to evaluate the model
def evaluate_model(model, dataloader):
	model.eval()
	running_loss = 0.0
	running_corrects = 0

	iterator = tqdm(dataloader, desc="Evaluation")
	# Iterate over data
	for batch_x, batch_car_make, batch_car_model in iterator:
		batch_x = batch_x.to(device)
		if car_make_bool:
			labels = batch_car_make.to(device)
		else:
			labels = batch_car_model.to(device)

		# Forward pass
		with torch.no_grad():
			outputs = model(batch_x)
			loss = criterion(outputs, labels)

		# Statistics
	running_loss += loss.item() * batch_x.size(0)
	running_corrects += accuracy(outputs, labels)

	epoch_loss = running_loss / len(dataloader.dataset)
	epoch_acc = running_corrects.double() / len(dataloader.dataset)

	print('Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))

In [ ]:
import torchvision.models as models
from torchvision.models import ResNet50_Weights

# Upload a pre-trained model to ImageNet
model = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
#model =  models.resnet50()   #NOT pretrained

# # Freeze all the layers in the network
# for param in model.parameters():
# 	param.requires_grad = False

# Get the number of features in the model
num_features = model.fc.in_features
print("Number of features:", num_features)
# Replace the final layer with a fully connected layer with the number of features in the model
if car_make_bool:
	model.fc = nnpo.Linear(num_features, MAKE_NUM)
else:
	model.fc = nnpo.Linear(num_features, MODELS_NUM)

# Move the model to the device
model = model.to(device)

# Define the loss function
criterion = nnpo.CrossEntropyLoss()
criterion = criterion.to(device)

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Define the learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# Load the checkpoint
model, optimizer, scheduler, start_epoch, best_acc = load_checkpoint(model, optimizer, scheduler, checkpoint_path)
print("Starting epoch:", start_epoch)

In [ ]:
# Train the model
model = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=12, start_epoch=start_epoch, best_acc=best_acc)

In [ ]:
# Evaluate the model on the test set
evaluate_model(model, test_loader)

In [ ]:
###############################
# load previous save model
###############################

# Load the model
model, optimizer, scheduler, start_epoch, best_acc = load_checkpoint(model, optimizer, scheduler, checkpoint_path)
model.eval()

# Predict the class of an image
image_path = os.path.join(test_dir, )

# Load the image
random_number = random.randint(0, len(test_dataset))
img, car_make, car_model = test_dataset[random_number]
img = denormalize(img, mean, std)
imshow(img)
print("Make:",car_make)
print("Model:",car_model)

print(get_names(mat_file, [car_make, car_model]))

# Predict the class of the image
pred = model(img.unsqueeze(0).to(device)).argmax().item()
print("Prediction:", pred, get_names(mat_file, [pred, 0]))
